In [70]:
from glob import glob

import asn1tools

In [71]:
files = glob("../**/*.asn", recursive=True)

def parseFiles(files):
  return asn1tools.parse_files(files)

parsed = parseFiles(files)
for f, p in zip(files, parsed):
  print(f)
  print(f"- {p}")

../ETSI-ITS-ASN1/cam_en302637_2/CAM-PDU-Descriptions.asn
- CAM-PDU-Descriptions
../ETSI-ITS-ASN1/cam_en302637_2/cdd/ITS-Container.asn
- ITS-Container


In [72]:
# f = "CAM-PDU-Descriptions"
f = "ITS-Container"

for k, v in parsed[f].items():
  print(k)
  if isinstance(v, dict):
    for e in v:
      print(f"- {e}")
  else:
    print(f"- {v}")

imports
types
- ItsPduHeader
- StationID
- ReferencePosition
- DeltaReferencePosition
- Longitude
- Latitude
- Altitude
- AltitudeValue
- AltitudeConfidence
- DeltaLongitude
- DeltaLatitude
- DeltaAltitude
- PosConfidenceEllipse
- PathPoint
- PathDeltaTime
- PtActivation
- PtActivationType
- PtActivationData
- AccelerationControl
- SemiAxisLength
- CauseCode
- CauseCodeType
- SubCauseCodeType
- TrafficConditionSubCauseCode
- AccidentSubCauseCode
- RoadworksSubCauseCode
- HumanPresenceOnTheRoadSubCauseCode
- WrongWayDrivingSubCauseCode
- AdverseWeatherCondition-ExtremeWeatherConditionSubCauseCode
- AdverseWeatherCondition-AdhesionSubCauseCode
- AdverseWeatherCondition-VisibilitySubCauseCode
- AdverseWeatherCondition-PrecipitationSubCauseCode
- SlowVehicleSubCauseCode
- StationaryVehicleSubCauseCode
- HumanProblemSubCauseCode
- EmergencyVehicleApproachingSubCauseCode
- HazardousLocation-DangerousCurveSubCauseCode
- HazardousLocation-SurfaceConditionSubCauseCode
- HazardousLocation-Obstac

In [73]:
types = parsed[f]["types"]

# for k in ["CAM", "CoopAwareness", "CamParameters", "HighFrequencyContainer", "GenerationDeltaTime"]:
#   print(k)
#   print(f"  {types[k]}")

In [74]:
type_types = set([v["type"] for v in types.values()])
print(type_types)

member_keys = set()
for v in types.values():
  if "members" in v:
    for m in v["members"]:
      if m is None:
        continue
      member_keys.update(list(m.keys()))
print(member_keys)

{'ENUMERATED', 'IA5String', 'BIT STRING', 'UTF8String', 'ProtectedZoneID', 'NumericString', 'SEQUENCE', 'OCTET STRING', 'SEQUENCE OF', 'BOOLEAN', 'INTEGER'}
{'type', 'named-numbers', 'size', 'optional', 'name', 'restricted-to'}


In [75]:
naive_types = {k: v for k, v in types.items() if v["type"] not in ("SEQUENCE", "SEQUENCE OF", "CHOICE")}
array_types = {k: v for k, v in types.items() if v["type"] == "SEQUENCE OF"}

print(list(naive_types.keys()))
print(list(array_types.keys()))

['StationID', 'Longitude', 'Latitude', 'AltitudeValue', 'AltitudeConfidence', 'DeltaLongitude', 'DeltaLatitude', 'DeltaAltitude', 'PathDeltaTime', 'PtActivationType', 'PtActivationData', 'AccelerationControl', 'SemiAxisLength', 'CauseCodeType', 'SubCauseCodeType', 'TrafficConditionSubCauseCode', 'AccidentSubCauseCode', 'RoadworksSubCauseCode', 'HumanPresenceOnTheRoadSubCauseCode', 'WrongWayDrivingSubCauseCode', 'AdverseWeatherCondition-ExtremeWeatherConditionSubCauseCode', 'AdverseWeatherCondition-AdhesionSubCauseCode', 'AdverseWeatherCondition-VisibilitySubCauseCode', 'AdverseWeatherCondition-PrecipitationSubCauseCode', 'SlowVehicleSubCauseCode', 'StationaryVehicleSubCauseCode', 'HumanProblemSubCauseCode', 'EmergencyVehicleApproachingSubCauseCode', 'HazardousLocation-DangerousCurveSubCauseCode', 'HazardousLocation-SurfaceConditionSubCauseCode', 'HazardousLocation-ObstacleOnTheRoadSubCauseCode', 'HazardousLocation-AnimalOnTheRoadSubCauseCode', 'CollisionRiskSubCauseCode', 'SignalViolat

In [76]:
def ansType2RosMsg(type_name, type_info):

  if type_name in naive_types or type_name in array_types:
    return

  print(type_name)
  type_type = type_info["type"]
  if type_type == "SEQUENCE":
    for member in type_info["members"]:
      if member is None:
        continue
      if member["type"] in naive_types:
        member_name = member["name"]
        member = naive_types[member["type"]]
        member["name"] = member_name
      if member["type"] in array_types:
        member_name = member["name"]
        member_name = member["name"]
        member = array_types[member['type']]
        member["name"] = member_name
        member["type"] = f"{member['element']['type']}[]"
      lines = []
      comment_lines = []
      lines.append(f"  {member['type']} {member['name']}")
      if "values" in member:
        for val in member["values"]:
          if val is None:
            continue
          k, v = val
          lines.append(f"  INTEGER {member['name'].upper()}_{k.upper().replace('-', '_')} = {v}")
      if "named-numbers" in member:
        for k, v in member["named-numbers"].items():
          lines.append(f"  {member['type']} {member['name'].upper()}_{k.upper().replace('-', '_')} = {v}")
      for k, v in member.items():
        if k not in ("type", "name", "values", "named-numbers", "element"):
          comment_lines.append(f"{k}: {v}")
      print()
      comment_lines = [f"  # {cl}" for cl in comment_lines]
      for cl in comment_lines:
        print(cl)
      for l in lines:
        print(l)
  elif type_type == "CHOICE":
    # {'type': 'CHOICE', 'members': [{'type': 'BasicVehicleContainerHighFrequency', 'name': 'basicVehicleContainerHighFrequency'}, {'type': 'RSUContainerHighFrequency', 'name': 'rsuContainerHighFrequency'}, None]}
    pass
  else:
    raise ValueError(f"Unhandled type {type_type}")

for k, v in types.items():
  ansType2RosMsg(k, v)
  print("----------------------------------------")

ItsPduHeader

  # restricted-to: [(0, 255)]
  INTEGER protocolVersion

  # restricted-to: [(0, 255)]
  INTEGER messageID
  INTEGER MESSAGEID_DENM = 1
  INTEGER MESSAGEID_CAM = 2
  INTEGER MESSAGEID_POI = 3
  INTEGER MESSAGEID_SPATEM = 4
  INTEGER MESSAGEID_MAPEM = 5
  INTEGER MESSAGEID_IVIM = 6
  INTEGER MESSAGEID_EV_RSR = 7
  INTEGER MESSAGEID_TISTPGTRANSACTION = 8
  INTEGER MESSAGEID_SREM = 9
  INTEGER MESSAGEID_SSEM = 10
  INTEGER MESSAGEID_EVCSN = 11
  INTEGER MESSAGEID_SAEM = 12
  INTEGER MESSAGEID_RTCMEM = 13

  # restricted-to: [(0, 4294967295)]
  INTEGER stationID
----------------------------------------
----------------------------------------
ReferencePosition

  # restricted-to: [(-900000000, 900000001)]
  INTEGER latitude
  INTEGER LATITUDE_ONEMICRODEGREENORTH = 10
  INTEGER LATITUDE_ONEMICRODEGREESOUTH = -10
  INTEGER LATITUDE_UNAVAILABLE = 900000001

  # restricted-to: [(-1800000000, 1800000001)]
  INTEGER longitude
  INTEGER LONGITUDE_ONEMICRODEGREEEAST = 10
  INTEGER LO